# Training Model YOLO
**Penulis:** Irfan Muzaki

**Update Terbaru:** 17 Mei 2025

**GitHub:** [Train and Deploy YOLO Models](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models)

# Pendahuluan

Notebook ini menggunakan [Ultralytics](https://docs.ultralytics.com/) to men-training model dataset sendiri

Ada Channel YouTube yang membahas tentang membuat model dan training YOLO untuk Object Detection, Klik Tautan di bawah ini. Credit: Evan Juras, EJ Technology Consultants

<p align=center>
<a href="https://youtu.be/r0RspiLG260" target="_blank"><img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/Train_YOLO_Thumbnail2.png" height="240"><br>
<i>Klik di sini untuk pergi ke tautan!!</i></a>
</p>




## Memastikan GPU NVIDIA Tersedia

*Rekomendasi GPU: T4*

Pastikan sudah menggunakan GPU dengan mengklik "Runtime" di atas kiri halaman ini (menu bar), lalu klik "Change runtime type", dan pilih GPU yang tersedia. Klik tombol di bawah ini untuk memastikan GPU tersedia untuk Training.

In [ ]:
!nvidia-smi

#1.&nbsp;Menyiapkan Dataset

Sebelum Training, pastikan folder sudah berbentuk seperi ini

- `images` folder yang berisi gambar dataset
- `labels` folder yang berisi label dalam forfmat YOLO (.txt)
- `classes.txt` daftar kelas yang ada
- `notes.json` info tambahan dari label-studio (bisa di abaikan saja)

Setelah folder berbentuk seperti ini, jadikan ke dalam bentuk archive `.zip`

Jika dataset diperoleh dari sumber lain (seperti Roboflow Universe) atau menggunakan alat lain untuk melabeli dataset, pastikan file-file tersebut sudah diatur dalam struktur folder seperti di atas dan di jadikan `.zip`.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/zipped-data-example.png" height=""><br>


# 2.&nbsp;Upload Gambar Dataset dan Pre-processing

## 2.1 Upload Gambar

Upload dataset yang telah dijadikan dalam bentuk `.zip` ke sini dengan mengklik gambar folder di samping kiri ini, bisa seret lepas, atau seperti gambar di bawah ini

<p>
<br>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/upload-colab-files.png" height="240">
</p>



**Atau bisa juga Upload dari Google Drive**

Upload dataset ke Google Drive pribadi Anda. Opsi ini sangat berguna jika Anda ingin membiarkan dataset berada di sini karena setiap Google Colab ini di restart, maka dataset Anda akan hilang. Jika Anda memiliki gambar berukuran lebih dari 50MB, sangat disarankan untuk menggunakan opsi ini.


Pertama, upload file `.zip` ke Google Drive Anda, dan salin folder tempat Anda menguploadnya. Ganti `MyDrive/..../..../(nama file dataset anda).zip` dengan jalur ke file zip Anda. (Contohnya, saya mengupload file zip ke folder bernama “dataset saya”, jadi saya akan menggunakan `MyDrive/dataset saya/(nama data set saya).zip` untuk jalurnya). Kemudian, jalankan blok kode berikut untuk memasang Google Drive Anda ke sesi Colab ini dan salin folder ke sistem file ini.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content #<<--------GANTI DENGAN NAMA PATH ANDA

## 2.2 Split Gambar ke Folder "train" dan "validation"

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/XXXXXXXXXXXXXXXXXX.zip -d /content/custom_data #<GANTI XXXXXXXXXXXXXXX dengan

Ultralytics memerlukan struktur folder tertentu untuk menyimpan data pelatihan untuk model. Folder root diberi nama “data”. Di dalamnya terdapat dua folder utama:

*   **train**: berisi gambar asli yang digunakan untuk melatih model. Dalam satu epoch pelatihan, setiap gambar dalam folder `train` diteruskan ke neural network. Algoritma pelatihan menyesuaikan bobot jaringan agar sesuai dengan data dalam gambar.

*   **validation**: Gambar-gambar di folder ini digunakan untuk memeriksa kinerja model pada akhir setiap epoch pelatihan.

Di setiap folder ini terdapat folder “images” dan folder “labels”, yang masing-masing menyimpan file gambar dan file annotation.


Di bawah ini adalah script Phyton yang dibuat oleh seseorang, dimana script akan otomatis membuat folder yang dibutuhkan, lalu 90% gambar dataset dipindahkan ke folder "train" dan 10% ke folder "validation" folder. Jalankan script dibawah ini untuk menjalankan.

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

# 3.&nbsp;Install Ultralytics untuk Training


In [ ]:
!pip install ultralytics

# 4.&nbsp;Configure Training


Ada satu langkah lagi sebelum kita menjalankan pelatihan: kita perlu membuat file YAML konfigurasi pelatihan Ultralytics. File ini menentukan lokasi data pelatihan dan validasi Anda, dan juga mendefinisikan kelas-kelas model. Contoh file konfigurasi model tersedia [di sini](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco128.yaml).

Jalankan blok kode di bawah ini untuk secara otomatis menghasilkan file konfigurasi `data.yaml`. Pastikan Anda memiliki file labelmap atau kelas yang terletak di `custom_data/classes.txt`. Jika Anda menggunakan Label Studio atau salah satu dataset yang sudah dibuat sebelumnya, seharusnya filenya sudah ada. Jika Anda mengumpulkan dataset dengan cara lain, Anda mungkin harus membuat file `classes.txt` secara manual (lihat [di sini] (https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/doc/classes.txt) untuk contoh formatnya).

In [ ]:

# Fungsi Python untuk membuat file konfigurasi data.yaml secara otomatis
# 1. Membaca file “classes.txt” untuk mendapatkan daftar nama kelas
# 2. Membuat dictionary data dengan jalur yang benar ke folder, jumlah kelas, dan nama-nama kelas
# 3. Menulis data dalam format YAML ke data.yaml


import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Membaca class.txt untuk mendapatkan nama-nama kelas
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Membuat dictionary data
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Menulis data ke file YAML
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Tentukan jalur ke class.txt dan jalankan fungsi
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

# 5.&nbsp;Training Model


## 5.1 Parameter Pelatihan

Sekarang data sudah diorganisir dan file konfigurasi telah dibuat, kita siap untuk memulai pelatihan. Pertama, ada beberapa parameter penting yang harus diputuskan. Kunjungi artikel tentang [Melatih Model YOLO Secara Lokal](https://www.ejtech.io/learn/train-yolo-models) untuk mempelajari lebih lanjut tentang parameter-parameter ini dan cara memilihnya.

**Arsitektur & ukuran model (`model`):**
Tersedia beberapa ukuran model YOLO11 yang bisa digunakan untuk pelatihan, termasuk `yolo11n.pt`, `yolo11s.pt`, `yolo11m.pt`, `yolo11l.pt`, dan `yolo11xl.pt`. Model yang lebih besar berjalan lebih lambat namun memiliki akurasi lebih tinggi, sementara model yang lebih kecil berjalan lebih cepat tetapi akurasinya lebih rendah.

Anda juga bisa melatih model YOLOv8 atau YOLOv5 dengan mengganti `yolo11` menjadi `yolov8` atau `yolov5`.

**Jumlah epoch (`epochs`)**
Dalam machine learning, satu “epoch” berarti satu kali siklus penuh melalui seluruh dataset pelatihan. Menentukan jumlah epoch berarti menentukan berapa lama model akan dilatih. Jumlah epoch terbaik tergantung pada ukuran dataset dan arsitektur model. Jika dataset anda memiliki kurang dari 200 gambar, 60 epoch adalah titik awal yang baik. Jika dataset anda memiliki lebih dari 200 gambar, titik awal yang baik adalah 40 epoch.

**Resolusi (`imgsz`)**
Resolusi sangat memengaruhi kecepatan dan akurasi model: model dengan resolusi rendah akan memiliki kecepatan lebih tinggi tetapi akurasi lebih rendah. Model YOLO biasanya dilatih dan digunakan dengan resolusi 640x640. Namun, jika ingin model berjalan lebih cepat atau bekerja dengan gambar beresolusi rendah, cobalah menggunakan resolusi yang lebih rendah seperti 480x480.


## 5.2 Mari Kita Training!


**Jalankan blok kode berikut untuk memulai pelatihan.**
Jika Anda ingin menggunakan model yang berbeda, jumlah epoch yang berbeda, atau resolusi yang berbeda, ubah nilai dari `model`, `epochs`, atau `imgsz`.



In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

**Algoritma pelatihan akan memproses gambar-gambar dalam direktori pelatihan dan validasi, lalu mulai melatih model.**
Di akhir setiap epoch pelatihan, program akan menjalankan model pada dataset validasi dan melaporkan nilai *mAP*, *precision*, dan *recall* yang dihasilkan.
Seiring pelatihan berjalan, *mAP* biasanya akan meningkat di setiap epoch.
Pelatihan akan berakhir setelah melewati jumlah epoch yang ditentukan oleh `epochs`.

> **CATATAN:** Pastikan pelatihan dibiarkan berjalan hingga selesai, karena ada proses *optimizer* yang dijalankan di akhir pelatihan untuk menghapus layer-layer yang tidak diperlukan dari model.

Model terbaik hasil pelatihan akan disimpan di:
`content/runs/detect/train/weights/best.pt`
Informasi tambahan tentang proses pelatihan disimpan dalam folder:
`content/runs/detect/train`
Termasuk file `results.png` yang menunjukkan perkembangan *loss*, *precision*, *recall*, dan *mAP* selama setiap epoch.



#6.&nbsp;Test Model

**Model telah selesai dilatih; sekarang saatnya mengujinya!**
Perintah di bawah ini akan menjalankan model pada gambar-gambar di folder validasi dan menampilkan hasilnya untuk 10 gambar pertama.
Ini adalah cara yang bagus untuk memastikan bahwa model kamu bekerja sesuai harapan.

Klik **Play** pada blok-blok di bawah ini untuk melihat bagaimana performa model kamu.



In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')



**Model seharusnya menggambar kotak di sekitar setiap objek yang terdeteksi dalam gambar.**
Jika model belum mendeteksi objek dengan baik, berikut beberapa tips yang bisa membantu:

1. Periksa ulang dataset kamu untuk memastikan tidak ada kesalahan label atau contoh yang saling bertentangan.
2. Tambahkan jumlah *epoch* saat pelatihan.
3. Gunakan model dengan ukuran lebih besar (misalnya `yolo11l.pt`).
4. Tambahkan lebih banyak gambar ke dataset pelatihan. Lihat video tentang [tips pengambilan gambar untuk dataset](https://www.youtube.com/watch?v=v0ssiOY6cfg) agar pelatihan jadi lebih akurat.

Anda juga bisa menjalankan model pada file video atau gambar lainnya dengan mengunggahnya ke notebook ini dan menggunakan perintah `!yolo detect predict` seperti di atas, di mana `source` diarahkan ke lokasi file video, gambar, atau folder gambar.
Hasil prediksinya akan disimpan di:
`runs/detect/predict`

Menggambar kotak pada gambar memang bagus, tapi itu sendiri belum terlalu berguna.
Selain itu, menjalankan model langsung di dalam notebook Colab juga kurang praktis.
Akan jauh lebih berguna jika kita bisa menjalankannya langsung di komputer lokal.

Lanjut ke bagian berikutnya untuk melihat cara mengunduh model hasil pelatihan dan menjalankannya di perangkat lokal anda.


#7.&nbsp;Deploy Model

**Sekarang model kustom anda sudah selesai dilatih, saatnya untuk diunduh dan digunakan dalam aplikasi**
Model YOLO dapat dijalankan di berbagai jenis perangkat keras, termasuk PC, sistem embedded, dan ponsel.
Ultralytics memudahkan proses konversi model YOLO ke berbagai format seperti `tflite`, `onnx`, dan lainnya, sehingga bisa digunakan di berbagai lingkungan.


## 7.1 Download Model YOLO

**Pertama, kompres dan unduh model hasil pelatihan dengan menjalankan blok kode di bawah ini.**

Kode ini akan membuat folder bernama `my_model`, memindahkan *model weights* ke dalamnya, dan mengganti nama file dari `best.pt` menjadi `my_model.pt`.
Folder tersebut juga akan menyertakan hasil pelatihan, sehingga kamu bisa merujuknya kembali nanti jika diperlukan.
Setelah itu, folder akan dikompres menjadi file `my_model.zip`.


In [ ]:
# Membuat folder "my_model" untuk menyimpan bobot model dan hasil
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip menjadi "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Jika tidak mau, download dari samping
from google.colab import files

files.download('/content/my_model.zip')


Selanjutnya, kita akan menggunakan model yang telah diunduh dan menjalankannya di perangkat lokal.
Bagian ini menyediakan panduan tentang cara men-deploy model YOLO di berbagai perangkat.

Seseorang telah menulis skrip Python sederhana, `yolo_detect.py`, yang menunjukkan cara memuat model, menjalankan *inference* pada sumber gambar, memproses hasil inference, dan menampilkan kotak di sekitar setiap kelas yang terdeteksi dalam gambar.
Skrip tersebut bisa kamu temukan di [tautan ini](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/yolo_detect.py).
Skrip ini memberikan contoh dasar penggunaan model YOLO dari Ultralytics di Python, dan bisa dijadikan titik awal untuk pengembangan aplikasi yang lebih kompleks.




### 7.2.1 Men-deploy di PC (Windows, Linux, atau macOS)

Cara termudah untuk menjalankan model Ultralytics di PC adalah dengan menggunakan **Anaconda**.
Anaconda membuat lingkungan virtual Python dan memudahkan anda untuk menginstal Ultralytics dan PyTorch.
Anaconda juga secara otomatis mengatur **CUDA** dan **cuDNN**, sehingga anda bisa mempercepat proses *inference* dengan GPU NVIDIA.

> **CATATAN:** Video YouTube (tautan akan ditambahkan) menunjukkan cara men-deploy model di PC. Video tersebut menjelaskan langkah-langkah berikut secara visual, jadi tontonlah jika kamu lebih suka panduan berbasis video.

---

**1. Unduh dan Instal Anaconda**

Kunjungi halaman unduh Anaconda di:
[https://anaconda.com/download](https://anaconda.com/download)
Klik tombol “skip registration”, lalu unduh paket untuk sistem operasi anda.
Setelah selesai mengunduh, jalankan instalernya dan ikuti langkah-langkah instalasi.
anda bisa menggunakan opsi default.

---

**2. Siapkan Lingkungan Virtual**

Setelah Anaconda terinstal, buka **Anaconda Prompt** dari Start Menu.
(Jika anda menggunakan macOS atau Linux, cukup buka terminal.)

Jalankan perintah berikut untuk membuat lingkungan Python baru dan mengaktifkannya:

```
conda create --name yolo-env1 python=3.12 -y
conda activate yolo-env1
```

Instal Ultralytics (yang juga akan menginstal pustaka seperti OpenCV-Python, Numpy, dan PyTorch):

```
pip install ultralytics
```

---

**3. Ekstrak Model yang Sudah Diunduh**

Ambil file `my_model.zip` yang sudah anda unduh di Langkah 7.1, lalu ekstrak ke sebuah folder di PC anda.
Di terminal Anaconda Prompt, masuk ke folder hasil ekstraksi:

```
cd path/to/folder
```

Ganti `path/to/folder` dengan jalur folder yang sesuai.

---

**4. Unduh dan Jalankan `yolo_detect.py`**

Unduh skrip `yolo_detect.py` ke dalam folder `my_model` dengan perintah berikut:

```
curl -o yolo_detect.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/yolo_detect.py
```

Oke! Sekarang kita siap menjalankan skrip.
Untuk menjalankan *inference* dengan model `yolov8s` pada kamera USB dengan resolusi 1280x720, jalankan:

```
python yolo_detect.py --model my_model.pt --source usb0 --resolution 1280x720
```

Sebuah jendela akan muncul menampilkan *live feed* dari webcam anda, dengan kotak deteksi pada setiap objek yang terdeteksi di setiap frame.

---

anda juga bisa menjalankan model pada file video, gambar tunggal, atau folder gambar.
Untuk melihat daftar lengkap argumen yang tersedia di `yolo_detect.py`, jalankan:

```
python yolo_detect.py --help
```

Atau kunjungi [README file](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/README.md) di GitHub.


